# Creating a Databunch for Basecalling

In [24]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [25]:
from fastai.basics import *

import jkbc.utils.experimentation as e
import jkbc.utils.files as f
import jkbc.utils.loss as l
import jkbc.utils.postprocessing as pop
import jkbc.utils.preprocessing as prep
import toml

## Constants

### Data

In [26]:
BLANK_ID       = pop.BLANK_ID
ALPHABET       = pop.ALPHABET # {0: '-', 1: 'A', 2: 'B'}
ALPHABET_VAL   = list(ALPHABET.values())
ALPHABET_STR   = ''.join(ALPHABET_VAL)
ALPHABET_SIZE  = len(ALPHABET_VAL)
WINDOW_SIZE    = 300
STRIDE         = 300
DIMENSIONS_OUT = 70 #Max label len?

### Model

In [27]:
DIMENSIONS_PREDICTION_OUT = 100     # DIMENSIONS_OUT*2-1
DROP_LAST    = False # SET TO TRUE IF IT FAILS ON LAST BATCH
## Add Model specific variables here

### Train/Predict

In [35]:
BS = 1024  # batch size
DEVICE = torch.device("cuda:0") #torch.device("cpu")
MODEL_NAME = "bonito_5000_10"

### Data path

In [10]:
BASE_DIR = Path("../..")
PATH_DATA = 'data/feather-files'
DATA_SET = f'Range0-5000-FixLabelLen{DIMENSIONS_OUT}'
FEATHER_FOLDER = BASE_DIR/PATH_DATA/DATA_SET

## Load data

In [11]:
# Read data from feather
data = f.read_data_from_feather_file(FEATHER_FOLDER)

# Convert to databunch
train_dl, valid_dl = prep.convert_to_dataloaders(data, split=.8, batch_size=BS, drop_last=DROP_LAST)
del data
databunch = DataBunch(train_dl, valid_dl, device=DEVICE)
del train_dl
del valid_dl

## Model

In [12]:
loss_func = l.ctc_loss(DIMENSIONS_PREDICTION_OUT, BS, ALPHABET_SIZE)

## TODO: MAKE MODEL

In [13]:
activations = {
    "relu": nn.ReLU,
    "leaky_relu": nn.LeakyReLU,
}


class Model(nn.Module):
    """
    Model template for QuartzNet style architectures

    https://arxiv.org/pdf/1910.10261.pdf
    """
    def __init__(self, config):
        super(Model, self).__init__()
        self.stride = config['block'][0]['stride'][0]
        self.alphabet = config['labels']['labels']
        self.features = config['block'][-1]['filters']
        self.encoder = Encoder(config)
        self.decoder = Decoder(self.features, len(self.alphabet))

    def forward(self, x):
        encoded = self.encoder(x)
        return self.decoder(encoded)


class Encoder(nn.Module):
    """
    Builds the model encoder
    """
    def __init__(self, config):
        super(Encoder, self).__init__()
        self.config = config

        features = self.config['input']['features']
        activation = activations[self.config['encoder']['activation']]()
        encoder_layers = []

        for layer in self.config['block']:
            encoder_layers.append(
                Block(
                    features, layer['filters'], activation,
                    repeat=layer['repeat'], kernel_size=layer['kernel'],
                    stride=layer['stride'], dilation=layer['dilation'],
                    dropout=layer['dropout'], residual=layer['residual'],
                    separable=layer['separable'],
                )
            )

            features = layer['filters']

        self.encoder = nn.Sequential(*encoder_layers)

    def forward(self, x):
        return self.encoder([x])


class TCSConv1d(nn.Module):
    """
    Time-Channel Separable 1D Convolution
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=False, separable=False):

        super(TCSConv1d, self).__init__()
        self.separable = separable

        if separable:
            self.depthwise = nn.Conv1d(
                in_channels, in_channels, kernel_size=kernel_size, stride=stride,
                padding=padding, dilation=dilation, bias=bias, groups=in_channels
            )

            self.pointwise = nn.Conv1d(
                in_channels, out_channels, kernel_size=1, stride=stride,
                dilation=dilation, bias=bias, padding=0
            )
        else:
            self.conv = nn.Conv1d(
                in_channels, out_channels, kernel_size=kernel_size,
                stride=stride, padding=padding, dilation=dilation, bias=bias
            )

    def forward(self, x):
        if self.separable:
            x = self.depthwise(x)
            x = self.pointwise(x)
        else:
            x = self.conv(x)
        return x


class Block(nn.Module):
    """
    TCSConv, Batch Normalisation, Activation, Dropout
    """
    def __init__(self, in_channels, out_channels, activation, repeat=5, kernel_size=1, stride=1, dilation=1, dropout=0.0, residual=False, separable=False):

        super(Block, self).__init__()

        self.use_res = residual
        self.conv = nn.ModuleList()

        _in_channels = in_channels
        padding = self.get_padding(kernel_size[0], stride[0], dilation[0])

        # add the first n - 1 convolutions + activation
        for _ in range(repeat - 1):
            self.conv.extend(
                self.get_tcs(
                    _in_channels, out_channels, kernel_size=kernel_size,
                    stride=stride, dilation=dilation,
                    padding=padding, separable=separable
                )
            )

            self.conv.extend(self.get_activation(activation, dropout))
            _in_channels = out_channels

        # add the last conv and batch norm
        self.conv.extend(
            self.get_tcs(
                _in_channels, out_channels,
                kernel_size=kernel_size,
                stride=stride, dilation=dilation,
                padding=padding, separable=separable
            )
        )

        # add the residual connection
        if self.use_res:
            self.residual = nn.Sequential(*self.get_tcs(in_channels, out_channels))

        # add the activation and dropout
        self.activation = nn.Sequential(*self.get_activation(activation, dropout))

    def get_activation(self, activation, dropout):
        return activation, nn.Dropout(p=dropout)

    def get_padding(self, kernel_size, stride, dilation):
        if stride > 1 and dilation > 1:
            raise ValueError("Dilation and stride can not both be greater than 1")
        return (kernel_size // 2) * dilation

    def get_tcs(self, in_channels, out_channels, kernel_size=1, stride=1, dilation=1, padding=0, bias=False, separable=False):
        return [
            TCSConv1d(
                in_channels, out_channels, kernel_size,
                stride=stride, dilation=dilation, padding=padding,
                bias=bias, separable=separable
            ),
            nn.BatchNorm1d(out_channels, eps=1e-3, momentum=0.1)
        ]

    def forward(self, x):
        _x = x[0]
        for layer in self.conv:
            _x = layer(_x)
        if self.use_res:
            _x += self.residual(x[0])
        return [self.activation(_x)]


class Decoder(Module):
    """
    Decoder
    """
    def __init__(self, features, classes):
        super(Decoder, self).__init__()
        self.layers = nn.Sequential(nn.Conv1d(features, classes, kernel_size=1, bias=True))

    def forward(self, x):
        x = self.layers(x[-1])
        return nn.functional.log_softmax(x.transpose(1, 2), dim=2)


In [14]:
config = toml.load("quartznet5x5.toml")
model = Model(config).to(device=DEVICE)

In [17]:
learner = Learner(databunch, model, loss_func=loss_func, path=".")

In [36]:
try:
    learner = learner.load(MODEL_NAME)
    print(f'Model weights loaded for: {MODEL_NAME}')
except:
    print('No model weights available')

Model weights loaded for: bonito_5000_10


In [19]:
learner.lr_find()
learner.recorder.plot(suggestion=True)

epoch,train_loss,valid_loss,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: 

In [ ]:
learner.fit_one_cycle(500, max_lr=learner.recorder.min_grad_lr)

In [ ]:
learner.save(MODEL_NAME)

In [20]:
x, (y, _) = databunch.one_batch()
x_device = x.to(device=DEVICE)
y_pred = model(x_device).detach().cpu().numpy()

In [37]:
r = range(min(5, DIMENSIONS_PREDICTION_OUT), min(DIMENSIONS_PREDICTION_OUT+1, 50)) #Range can't contain values larger than PRED_OUT_DIM

for index in [0, BS-1]:
    actual = pop.convert_idx_to_base_sequence(y[index], ALPHABET_VAL)
    prediction = y_pred[index]
    for pred, beam, error in e.get_stats(prediction, actual, ALPHABET_STR, r):
        print(pred, beam, error.error)
    print('')

CGACATCGAGGTGCCAAACCTCC
CTGCAGGAT 5 0.7826086956521738
CTGCAGAT 6 0.7391304347826086
CTGCATGAT 7 0.6956521739130435
CTGCATGATA 8 0.6521739130434783
CATGCATGATA 9 0.782608695652174
GTGCATGATA 10 0.8260869565217391
CGTGCATGATA 11 0.782608695652174
CGTGCATGATA 12 0.782608695652174
CTGCATGATA 13 0.6521739130434783
GTGCATGATA 14 0.8260869565217391
CGTGCATGATA 15 0.782608695652174
GTGCATGATA 16 0.8260869565217391
CGTGCATGGTA 17 0.8695652173913043
CGTGCATGGAT 18 0.782608695652174
CGTGCATGGAT 19 0.782608695652174
CGTGCATGGAT 20 0.782608695652174
CTGCATGGAT 21 0.6956521739130435
CTGCATGGAT 22 0.6956521739130435
CGCACATGGAT 23 0.6521739130434782
CGTGCATGGAT 24 0.782608695652174
CGTGCATGGAT 25 0.782608695652174
CGTGCATGGAT 26 0.782608695652174
CGTGCATGGAT 27 0.782608695652174
CGTGCATGGAT 28 0.782608695652174
CGTGCATGGAT 29 0.782608695652174
CGTGCATGGAT 30 0.782608695652174
CGTGCATGGAT 31 0.782608695652174
CGTGCATGGAT 32 0.782608695652174
CGTGCATGGAT 33 0.782608695652174
CGTGCATGGAT 34 0.782608695

In [23]:
# Run assemble only if data is not fetched from featherfile
# TODO: Assembled should not be on a batch, but instead on a complete signal
if STRIDE != WINDOW_SIZE:
    decoded = pop.decode(y_pred, ALPHABET_STR, beam_size=15)
    assembled = pop.assemble(decoded, WINDOW_SIZE, STRIDE, ALPHABET)
    print('Assembled:', assembled)